In [1]:
import requests
from bs4 import BeautifulSoup
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
import time

# ✅ 设置 Moonshot API Key（请替换为你自己的）
os.environ["OPENAI_API_KEY"] = "sk-klVo5anJzPqeluB6cwezzDx13bdZq9vLjZi9TxQNK5K4S3xv"

# ✅ 初始化 Moonshot 模型
llm = ChatOpenAI(
    base_url="https://api.moonshot.cn/v1",
    api_key=os.environ["OPENAI_API_KEY"],
    model="moonshot-v1-128k",
    temperature=0.4
)

# ✅ 获取新闻页面内容
import requests
url = "https://finance.eastmoney.com/"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
response.encoding = 'utf-8'

soup = BeautifulSoup(response.text, 'html.parser')

# ✅ 提取新闻标题和正文内容
articles = soup.find_all('a', title=True)
news_data = []

for article in articles:
    title = article.get('title')
    spans = article.find_all_next('span', class_='info')
    content = " ".join(span.get_text(strip=True) for span in spans)
    news_data.append(f"标题：{title}，内容：{content}")

# ✅ 构建 Prompt
news_block = "\n".join(news_data[:10])  # 取前5条新闻
prompt_template = PromptTemplate(
    input_variables=["news_block"],
    template="""
你是一位金融市场分析师，擅长从新闻中提取股市热点信息。以下是若干财经新闻内容，请你：
1. 识别涉及的热点行业或板块，并提取日期、涨停股、资金流向等信息；
2. 总结每个板块的表现（如涨跌幅、市场原因、政策影响等）；
3. 提取相关新闻摘要，整合政策、宏观经济变化等；
4. 汇总相关股票表现。

输出格式：
- 热点板块
- 相关新闻
- 相关股票

以下是原始新闻内容：
{news_block}
"""
)

# ✅ 生成摘要
chain = LLMChain(llm=llm, prompt=prompt_template)
summary = chain.invoke({"news_block": news_block})

# ✅ 输出结果
print("\n===== 新闻摘要结果 =====\n")
print(summary["text"] if isinstance(summary, dict) else summary)


C:\Users\Ji Xuanchi\AppData\Local\Temp\ipykernel_23344\2942766839.py:13: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
C:\Users\Ji Xuanchi\AppData\Local\Temp\ipykernel_23344\2942766839.py:61: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)



===== 新闻摘要结果 =====

- 热点板块
  1. 家用电器行业：领涨，荣泰健康涨停，海信视像涨9.21%，美的集团涨超4%。
  2. 人形机器人行业：金沙江创投退出早期具身智能项目，板块大跌。
  3. 特别国债：财政部将发行首批特别国债5000亿元，支持国有大型商业银行补充核心一级资本。

- 相关新闻
  1. 李稻葵：耐心资本培育企业需有坐十年冷板凳的思想准备，从融资和培养企业两方面要更加有耐心。
  2. 易方达基金年报显示，面对当前宏观环境，整体思路逐步从布局内需的左侧转变为降低估值水平，等待内需达到再平衡。
  3. 财政部将发行首批特别国债5000亿元，积极支持国有大型商业银行补充核心一级资本。
  4. 全球金融市场动荡，日韩股市重挫，欧美股指期货全线走低，引发全球资金的避险情绪升温。
  5. 美的集团年报亮相，业绩增长的同时，推出了每10股派35元的现金分红预案，拟回购不低于50亿元，不超过100亿元。
  6. 金沙江创投主管合伙人朱啸虎表示，旗下天使基金在过去几年投过一些早期的具身智能项目，而最近几个月都在退出这些项目。

- 相关股票
  1. 美的集团（000333.SZ）：年报业绩增长，推出大额现金分红预案和回购预案，股价涨超4%。
  2. 荣泰健康：涨停。
  3. 海信视像：涨9.21%。
  4. 小米集团（01810.HK）：美的集团清仓持有的小米集团股票，套现9.01亿元。
